> **Data Mining Programming Project**:
>
> - Konstantinos Konstantinidis AEM:2546 konkonstant@uth.gr
> - Nikolaos Stavrinos AEM:2631 nstavrinos@uth.gr

In [1]:
# Install Libraries
!pip install textblob
!pip install tweepy
!pip install wordcloud
!pip install langdetect

# Import Libraries

In [2]:
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import re
import string

from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report
from collections import Counter

# Download all the lexicons needed

In [3]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tweeter Api

In [4]:
# Authentication
consumerKey = "xebmbbEHiMzamPFMBhoEkW8N8"
consumerSecret = "irADKXxRXLoDlWP90DulNUN9kVQr7pfICTQlPA4BRbhUu9PZTj"
accessToken = "3053418828-Kqwl2veNMbb9hpbJ2Zdax8tVm6V9zBfRTGtB7Bt"
accessTokenSecret = "q6ZYSCCPlwhOoxV2QBCSqjRT0GOS941qCQQMbJWoy5wuk"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

# Sentiment Analysis without data preprocessing
## Ask user for topic and word count input

In [5]:

def percentage(part,whole):
    return 100 * float(part)/float(whole) 

keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = int(input ("Please enter how many tweets to analyze: "))


tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive  = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

for tweet in tweets:
    
    #print(tweet.text)
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1

    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

Please enter keyword or hashtag to search: data mining
Please enter how many tweets to analyze: 1500


TweepError: Twitter error response: status code = 429

# Number of Tweets (Total, Positive, Negative, Neutral)

In [ ]:
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

In [ ]:
tweet_list

# PieChart of current data

In [ ]:
labels = ['Positive ['+str(positive)+'%]' , 'Neutral ['+str(neutral)+'%]','Negative ['+str(negative)+'%]']
sizes = [positive, neutral, negative]
colors = ['green', 'blue','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use('default')
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword=  "+keyword+"" )
plt.axis('equal')
plt.show()

# Deleting duplicates 

In [ ]:
tweet_list.drop_duplicates(inplace = True)

# Extracting text values

In [ ]:
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
tw_list

# Cleaning Text (RT, URL ,Emoji etc)

In [ ]:
#Removing RT, Url,Emojis etc
remove_rt = lambda x: re.sub(r'RT|\@\w+','', x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()

tw_list.head(10)

# Removing Punctuation

In [ ]:
def remove_punct(text):
    # Remove numbers
    text = re.sub('[0-9]+', '', text)
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    return text

tw_list['punct'] = tw_list["text"].apply(lambda x: remove_punct(x))

# Appliyng tokenization

In [ ]:
def tokenization(text):
   
    text = word_tokenize(text)
    return text

tw_list['tokenized'] = tw_list['punct'].apply(lambda x: tokenization(x))

# Removing stopwords

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: remove_stopwords(x))

# Applying Stemmer

In [ ]:
#Appliyng Stemmer
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: stemming(x))

# Calculating Negative, Positive, Neutral and Compound values after processing

In [ ]:
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

tw_list.head(10)

# Creating new data frames for all sentiments (positive, negative and neutral)


In [ ]:
tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

# Function to count values of sentiments

In [ ]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [ ]:
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

# Create Pie Chart for processed data

In [ ]:
pichart = count_values_in_column(tw_list,"sentiment")
labels = [  'Neutral ['+str(pichart["Percentage"][0])+'%]','Positive ['+str(pichart["Percentage"][1])+'%]','Negative ['+str(pichart["Percentage"][2])+'%]']
sizes = pichart["Percentage"]
colors = [ 'blue','green','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use('default')
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword=  "+keyword+"" )
plt.axis('equal')
plt.show()

# Function to Create cloud of words

In [ ]:
def create_wordcloud(text):
    mask = np.array(Image.open("cloud.png"))
    stopwords = set(STOPWORDS)
    wc = WordCloud(background_color="white",
                  mask = mask,
                  max_words=1000,
                  stopwords=stopwords,
                  repeat=True)
    wc.generate(str(text))
    wc.to_file("wc.png")
    path="wc.png"
    display(Image.open(path))

### Creating wordcloud for all tweets


In [ ]:
create_wordcloud(tw_list["text"].values)

### Creating wordcloud for positive sentiment

In [ ]:
create_wordcloud(tw_list_positive["text"].values)

### Creating wordcloud for negative sentiment


In [ ]:
create_wordcloud(tw_list_negative["text"].values)

### Creating wordcloud for neutral sentiment

In [ ]:
create_wordcloud(tw_list_neutral["text"].values)

## Calculating tweet's lenght and word count


In [ ]:
tw_list['text_len'] = tw_list['text'].astype(str).apply(len)
tw_list['text_word_count'] = tw_list['text'].apply(lambda x: len(str(x).split()))

In [ ]:
round(pd.DataFrame(tw_list.groupby("sentiment").text_len.mean()),2)

In [ ]:
round(pd.DataFrame(tw_list.groupby("sentiment").text_word_count.mean()),2)

In [ ]:
tw_list.head()

# Machine Learning Analysis

In [ ]:
#Analyzer for CuntVectorizer
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

#converting the sentiment values to 0,1,2 
y=tw_list['sentiment']
sentiment_ordering = ['positive', 'neutral', 'negative']
y = y.apply(lambda x: sentiment_ordering.index(x))

#vectorazing the text values
cv = CountVectorizer(analyzer=clean_text) 
X = cv.fit_transform(tw_list['text'])


#splittig the data set into training and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)


#Naive Bayes Model
NB_model=MultinomialNB()
NB_model.fit(X_train,y_train)

y_pred_nb=NB_model.predict(X_test)

cf=classification_report(y_test,y_pred_nb)
print("NB",cf)


#Logistic Regression Model
LR_model = LogisticRegression(solver='lbfgs')
LR_model.fit(X_train, y_train)
y_predict_lr = LR_model.predict(X_test)

cf=classification_report(y_test, y_predict_lr)
print("LR",cf)


#SVM Model
SVM_model = svm.SVC()
SVM_model.fit(X_train, y_train)
y_predict_svm = SVM_model.predict(X_test)

cf=classification_report(y_test,y_predict_svm)
print("SVM",cf)

# Most Used Words

## Horizontal chart for word count

In [ ]:
def ybarchart(n_trigrams):
    labels = []
    values = []


    for a_tuple in n_trigrams:

        labels.append(a_tuple[0])
        values.append(a_tuple[1])

    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.barh(x - width/2, values, width)

    ax.set_ylabel('Words')
    ax.set_title('Common Words Found in Tweets (Without Stop or Collection Words)')
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    

    fig.tight_layout()

    plt.show()

## Function to find word correlation in same tweet

In [ ]:
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

## 1 word correlation

In [ ]:
countdf = get_top_n_gram(tw_list['text'],(1,1),10)
ybarchart(countdf[1:11])
countdf[1:11]

## 2 word correlation

In [ ]:
#n2_bigram
n2_bigrams = get_top_n_gram(tw_list['text'],(2,2),10)
ybarchart(n2_bigrams)
n2_bigrams

## 3 word correlation

In [ ]:
n3_trigrams = get_top_n_gram(tw_list['text'],(3,3),10)
ybarchart(n3_trigrams)
n3_trigrams

# Search a specific word in the tweets and with what other words it is used

In [ ]:
search_word = input("Please enter a word to search within the tweets: ")
count_search = Counter()

for  tweet in tw_list['nonstop']:
    terms_only = [term for term in tweet]
    
    if search_word in terms_only:
        count_search.update(terms_only)
print("Co-occurrence for %s:" % search_word)
print(count_search.most_common(10))